In [4]:
pip install pgmpy

  Using cached pgmpy-0.1.25-py3-none-any.whl.metadata (6.4 kB)
  Using cached torch-2.3.0-cp310-cp310-win_amd64.whl.metadata (26 kB)
  Using cached statsmodels-0.14.2-cp310-cp310-win_amd64.whl.metadata (9.5 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
Using cached pgmpy-0.1.25-py3-none-any.whl (2.0 MB)
Using cached statsmodels-0.14.2-cp310-cp310-win_amd64.whl (9.8 MB)
Using cached torch-2.3.0-cp310-cp310-win_amd64.whl (159.8 MB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

model = BayesianNetwork([('Burglary', 'Alarm'), ('Earthquake', 'Alarm'), ('Alarm', 'JohnCalls'), ('Alarm', 'MaryCalls')])

cpd_burglary = TabularCPD(variable='Burglary', variable_card=2, values=[[0.999], [0.001]])
cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2, values=[[0.998], [0.002]])
cpd_alarm = TabularCPD(variable='Alarm', variable_card=2,
                       values=[[0.05, 0.94, 0.29, 0.999],
                               [0.95, 0.06, 0.71, 0.001]],
                       evidence=['Burglary', 'Earthquake'], evidence_card=[2, 2])
cpd_john_calls = TabularCPD(variable='JohnCalls', variable_card=2,
                            values=[[0.95, 0.05],
                                    [0.05, 0.95]],
                            evidence=['Alarm'], evidence_card=[2])
cpd_mary_calls = TabularCPD(variable='MaryCalls', variable_card=2,
                            values=[[0.80, 0.01],
                                    [0.20, 0.99]],
                            evidence=['Alarm'], evidence_card=[2])

model.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_john_calls, cpd_mary_calls)

assert model.check_model()

inference = VariableElimination(model)
probability = inference.query(variables=['Earthquake'], evidence={'Alarm': True, 'JohnCalls': True, 'MaryCalls': True})

print("Probability of Earthquake given John and Mary call:")
print(probability)

Probability of Earthquake given John and Mary call:
+---------------+-------------------+
| Earthquake    |   phi(Earthquake) |
+===============+===================+
| Earthquake(0) |            0.9999 |
+---------------+-------------------+
| Earthquake(1) |            0.0001 |
+---------------+-------------------+
